# Imports

In [1]:
from sklearn.metrics import classification_report
from speech_models import speech_logistic_regression
from process_dataset import speech_features, text_features
from ensemble import SpeechTextEnsemble, StackEnsemble, VoteEnsemble, BlendEnsemble
import numpy as np
import pandas as pd
import pickle

/home/rafid/Documents/github/CSE400-NLP/env/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Methods

In [2]:
def check_accuracy(model, data_type, save_name=None):
    if data_type == 'text':
        x_train, x_test, y_train, y_test = text_features.get_train_test()
    else:
        x_train, x_test, y_train, y_test = speech_features.get_train_test() 
    
    model.fit(x_train, y_train)

    if save_name != None:
        model.save(save_name)

    results = model.predict(x_test)
    print(classification_report(y_test, results))

def cross_validate(model, data_type, cv, save_name=None):
    if data_type == 'text':
        x, y = text_features.get_data()
    else:
        x, y = speech_features.get_data()

    if save_name != None:
        model.save(save_name)

    scoring = {'accuracy': 'accuracy',
           'f1_macro': 'f1_macro',
           'precision_macro': 'precision_macro',
           'recall_macro' : 'recall_macro'}

    scores = model.cross_validate(x, y, cv=cv, scoring=scoring)
    print('Accuracy: ', np.mean(scores['test_accuracy']))
    print('F1 Macro: ', np.mean(scores['test_f1_macro']))
    print('Precision Macro: ', np.mean(scores['test_precision_macro']))
    print('Recall Macro: ', np.mean(scores['test_recall_macro']))


# Main

In [3]:
meta_cls = speech_logistic_regression.get_logistic_regression()


In [4]:
# speech_model = BlendEnsemble(meta_cls=meta_cls, data_type='speech')
speech_model = StackEnsemble(meta_cls=meta_cls, data_type='speech')
# speech_model = VoteEnsemble(type='soft', data_type='speech')
# speech_model = VoteEnsemble(type='hard', data_type='speech')

In [5]:
# text_model = BlendEnsemble(meta_cls=meta_cls, data_type='text')
text_model = StackEnsemble(meta_cls=meta_cls, data_type='text')
# text_model = VoteEnsemble(type='soft', data_type='text')
# text_model = VoteEnsemble(type='hard', data_type='text')

In [6]:
combined_model = SpeechTextEnsemble(fit_bases=False)
x_train_s, x_test_s, y_train_s, y_test_s = speech_features.get_train_test()
x_train_t, x_test_t, y_train_t, y_test_t = text_features.get_train_test()


In [7]:
result = combined_model.predict(x_test_s, x_test_t)


# Result

In [8]:
print(classification_report(y_test_s, result))

              precision    recall  f1-score   support

         ang       0.76      0.87      0.81       594
         hap       0.79      0.75      0.77       314
         neu       0.66      0.54      0.60       344
         sad       0.78      0.77      0.78       224

    accuracy                           0.75      1476
   macro avg       0.75      0.73      0.74      1476
weighted avg       0.75      0.75      0.75      1476

